In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\0\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\0\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetMobile\0\NASNetMobile.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[96.0, 47.0, 80.0, 20.0, 37.0, 10.0, 96.0, 47.0, 13.0, 55.0, 84.0, 10.0, 66.0, 35.0, 84.0, 52.0, 37.0, 95.0, 55.0, 45.0, 95.0, 35.0, 52.0, 16.0, 86.0, 40.0, 35.0, 45.0, 96.0, 95.0, 86.0, 47.0, 37.0, 66.0, 4.0, 52.0, 7.0, 45.0, 100.0, 95.0, 13.0, 32.0, 80.0, 66.0, 86.0, 80.0, 80.0, 20.0, 47.0, 84.0, 45.0, 95.0, 47.0, 93.0, 16.0, 96.0, 84.0, 52.0, 40.0, 100.0, 4.0, 55.0, 13.0, 55.0, 100.0, 37.0, 80.0, 86.0, 7.0, 45.0, 13.0, 93.0, 66.0, 86.0, 13.0, 37.0, 7.0, 52.0, 100.0, 37.0, 86.0, 93.0, 95.0, 16.0, 96.0, 30.0, 93.0, 100.0, 80.0, 45.0, 47.0, 30.0, 7.0, 37.0, 95.0, 84.0, 47.0, 55.0, 37.0, 86.0, 40.0, 45.0, 32.0, 23.0, 100.0, 66.0, 7.0, 40.0, 35.0, 96.0, 93.0, 37.0, 45.0, 40.0, 37.0, 96.0, 55.0, 84.0, 66.0, 52.0, 96.0, 37.0, 7.0, 7.0, 10.0, 37.0, 7.0, 37.0, 47.0, 20.0, 23.0, 10.0, 16.0, 16.0, 45.0, 13.0, 80.0, 100.0, 23.0, 13.0, 7.0, 35.0, 80.0, 20.0, 96.0, 23.0, 40.0, 95.0, 66.0, 66.0, 23.0, 95.0, 7.0, 47.0, 84.0, 16.0, 10.0, 52.0, 13.0, 80.0, 96.0, 84.0, 66.0, 96.0, 37.0, 84.0, 32.0, 40

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetMobile = tf.keras.applications.NASNetMobile(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetMobile.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetMobile(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

19993432/19993432 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (G  (None, 1056)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1057      
                                                                 
Total params: 4,270,773
Trainable params: 4,234,035
Non-trainable params: 36,738
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetMobile_loss_0.txt',history.history['loss'])
#要改
np.savetxt('NASNetMobile_valloss_0.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Thu Apr  6 17:33:07 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 66.2527

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetMobile.ckpt\assets


591/591 [==============================] - 193s 274ms/step - loss: 66.2527 - val_loss: 763.3864
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 2.5564

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetMobile.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetMobile.ckpt\assets


591/591 [==============================] - 220s 373ms/step - loss: 0.0386 - val_loss: 0.0552
Epoch 82/300
591/591 [==============================] - 133s 225ms/step - loss: 0.0391 - val_loss: 0.1100
Epoch 83/300
591/591 [==============================] - 133s 224ms/step - loss: 0.0528 - val_loss: 0.0621
Epoch 84/300
591/591 [==============================] - 133s 224ms/step - loss: 0.0498 - val_loss: 0.1174
Epoch 85/300
591/591 [==============================] - 132s 223ms/step - loss: 0.1743 - val_loss: 0.0747
Epoch 86/300
591/591 [==============================] - 132s 223ms/step - loss: 0.0372 - val_loss: 0.1219
Epoch 87/300
591/591 [==============================] - 132s 224ms/step - loss: 0.0530 - val_loss: 0.0798
Epoch 88/300
591/591 [==============================] - 132s 224ms/step - loss: 0.0345 - val_loss: 0.1120
Epoch 89/300
591/591 [==============================] - 133s 225ms/step - loss: 0.0411 - val_loss: 0.1093
Epoch 90/300
591/591 [==============================] - 132

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetMobile.ckpt\assets


591/591 [==============================] - 226s 382ms/step - loss: 0.0371 - val_loss: 0.0515
Epoch 153/300
591/591 [==============================] - 141s 238ms/step - loss: 0.0307 - val_loss: 0.0823
Epoch 154/300
591/591 [==============================] - 138s 233ms/step - loss: 0.0331 - val_loss: 0.0664
Epoch 155/300
591/591 [==============================] - 138s 233ms/step - loss: 0.0279 - val_loss: 0.0900
Epoch 156/300
591/591 [==============================] - 139s 235ms/step - loss: 0.0345 - val_loss: 0.0896
Epoch 157/300
591/591 [==============================] - 137s 232ms/step - loss: 0.0285 - val_loss: 0.0682
Epoch 158/300
591/591 [==============================] - 139s 234ms/step - loss: 0.0316 - val_loss: 0.1041
Epoch 159/300
591/591 [==============================] - 139s 235ms/step - loss: 0.0264 - val_loss: 0.0682
Epoch 160/300
591/591 [==============================] - 138s 233ms/step - loss: 0.0502 - val_loss: 0.1221
Epoch 161/300
591/591 [============================